## Research Notebook for 01 XAU Bot

This notebook looks into a simple SMAcrossover strategy using backtesting.py libray and mt5 as a data feed. the following are the steps that are applied:

1. get data from mt5 using settings of desired asset and timeframe 
2. using class from backtesting.py, backtest the strategy 
3. optimize the strategy for the best parameters 
4. visualize using graphical methods. 






In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cycler
%matplotlib inline

In [2]:
def plt_dark():
       colors = cycler('color',
                     ['#669FEE', '#66EE91', '#9988DD',
                     '#EECC55', '#88BB44', '#FFBBBB'])
       plt.rc('figure', facecolor='#313233')
       plt.rc('axes', facecolor="#313233", edgecolor='none',
              axisbelow=True, grid=True, prop_cycle=colors,
              labelcolor='gray')
       plt.rc('grid', color='474A4A', linestyle='solid')
       plt.rc('xtick', color='gray')
       plt.rc('ytick', direction='out', color='gray')
       plt.rc('legend', facecolor="#313233", edgecolor="#313233")
       plt.rc("text", color="#C9C9C9")
       plt.rc('figure', facecolor='#313233')
plt_dark()

In [13]:

# Settings 
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
start_date = datetime(2023, 8, 1)
end_date = datetime.now()

# Function 
def GetMTData():
    mt5.initialize()
    bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
    bars['time'] = pd.to_datetime(bars['time'], unit='s')
    bars.columns = bars.columns.str.capitalize()
    return bars

df  = GetMTData()
# xauusd.tail()

class SMAcross(Strategy):
    
    n1 = 20
    n2 = 50
    
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)
        
        
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
        

bt = Backtest(df, SMAcross, cash=100000, commission= 0.002, exclusive_orders=True)
            
output = bt.run()

output



    
    

C:\Users\User\AppData\Local\Temp\ipykernel_30652\2166583085.py:36: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SMAcross, cash=100000, commission= 0.002, exclusive_orders=True)


Start                                     0.0
End                                    3614.0
Duration                               3614.0
Exposure Time [%]                   97.538036
Equity Final [$]                 105074.47576
Equity Peak [$]                  109187.85786
Return [%]                           5.074476
Buy & Hold Return [%]               10.770687
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.305562
Avg. Drawdown [%]                   -0.675916
Max. Drawdown Duration                 1946.0
Avg. Drawdown Duration                   79.0
# Trades                                 73.0
Win Rate [%]                        36.986301
Best Trade [%]                       6.827271
Worst Trade [%]                      -1.68687
Avg. Trade [%]                    

In [14]:
bt.plot()


Row(id='4891', ...)

In [15]:
optim = bt.optimize(n1 = range(20, 130, 10),
                    n2 = range (20, 130, 10),
                    constraint = lambda x: x.n2 - x.n1 >= 5,
                    maximize = 'Return [%]')
bt.plot()

  0%|          | 0/10 [00:00<?, ?it/s]

Row(id='5771', ...)

In [16]:
optim

Start                                     0.0
End                                    3614.0
Duration                               3614.0
Exposure Time [%]                   90.041494
Equity Final [$]                  118620.2061
Equity Peak [$]                  119490.52492
Return [%]                          18.620206
Buy & Hold Return [%]               10.770687
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.602632
Avg. Drawdown [%]                   -0.547957
Max. Drawdown Duration                 1573.0
Avg. Drawdown Duration              54.101695
# Trades                                 23.0
Win Rate [%]                        43.478261
Best Trade [%]                       7.948826
Worst Trade [%]                     -1.143578
Avg. Trade [%]                    